In [1]:
import pandas as pd
from predict import get_representations
from tqdm import tqdm
import warnings
import os
from esm import pretrained, FastaBatchedDataset
import torch

In [2]:
def remap_aa(seq, key, value):
    if key in seq:
        seq = seq.replace(key, value)
    return seq

def remap_aas(seq, map=None):
    if map is None:
        map = {'B': 'N', 
               'Z': 'Q', 
               'U': 'C',
               'O': 'K',
               'J': 'L', 
               'X': '', 
               '*': ''}
    for key, value in map.items():
        seq = remap_aa(seq, key, value)
    return seq

In [3]:
%%time
seq_assemblies = pd.read_csv('../data/interim/seq_assemblies.csv', 
                             names=['product_accession', 'seq_id', 'seq', 'assembly'])

CPU times: user 5min 7s, sys: 30 s, total: 5min 37s
Wall time: 5min 37s


In [4]:
unique_assemblies = seq_assemblies['assembly'].drop_duplicates()
selected_assemblies = unique_assemblies.sample(n=500, random_state=7)

In [4]:
selected_seq_assemblies = seq_assemblies.loc[seq_assemblies['assembly'].isin(selected_assemblies)].reset_index(drop=True)

In [5]:
unique_seqs = selected_seq_assemblies[['product_accession', 'seq']].drop_duplicates()
len(unique_seqs)

1999032

In [7]:
unique_seqs_fa = '../data/interim/refseq_500_unique_seqs.faa'

In [23]:
with open(unique_seqs_fa, 'w') as f:
    for _, row in tqdm(unique_seqs.iterrows(), 
                       total=len(unique_seqs), position=0):
        print('>' + row['product_accession'], file=f)
        print(remap_aas(row['seq']), file=f)

100%|██████████| 1999032/1999032 [01:25<00:00, 23431.48it/s]


In [5]:
selected_assemblies.to_csv('../data/interim/refseq_500_selected_assemblies.csv', 
                           index=False)